In [2]:
import pysam
import numpy as np
import pandas as pd

In [3]:
duplex_path = "test_duplex.bam"
fiberseq_path = "test_fiberseq.bam"

In [7]:
with pysam.AlignmentFile(fiberseq_path, "rb") as bam:
    # Iterate over alignments
    for read in bam:
        if read.modified_bases_forward:
            print( read.query_name, read.modified_bases_forward )

m54119U_221124_201434/41879687/ccs {('T', 1, 'a'): [(418, 246), (714, 246), (1821, 247), (3347, 250), (3982, 250), (4351, 248), (5288, 252), (7267, 246), (7681, 251), (8073, 251), (8076, 252), (8257, 252), (8261, 248), (8290, 244), (8611, 245), (8623, 250), (8628, 249), (8897, 249), (8900, 251), (9380, 253), (9385, 254), (9387, 247), (9556, 244), (9596, 250), (10585, 244), (10588, 246), (11538, 248), (12322, 246), (12342, 252), (12346, 245), (12349, 250), (12661, 247), (12670, 248), (12673, 246), (14757, 246), (15686, 246), (16955, 247), (16969, 249), (17184, 249), (20903, 250), (20905, 249), (20908, 247), (20909, 253), (20914, 246), (20925, 244), (20929, 252), (20936, 253), (20938, 252), (20952, 245), (20963, 254), (20964, 253), (20967, 254), (20970, 254), (20974, 249), (20999, 251), (21006, 248), (21025, 249), (21032, 248), (21035, 245), (21649, 244), (22497, 245), (22616, 247), (22971, 248)], ('A', 0, 'a'): [(1594, 247), (2905, 248), (4631, 245), (5766, 249), (7679, 253), (7684, 252